In [ ]:
import os, shutil, yaml
import papermill as pm

In [ ]:
path_to_default_config = 'config.yml'

In [ ]:
# specify sessions to analyze (Animal ID, Date, Area) 
sessions_to_analyze = [
    [ 'calcium','20190615', 'S1'],
    [ 'calcium','20190617', 'S1'],
    [ 'calcium','20190618', 'S1'],
    [ 'calcium','20190624', 'S1'],
    [ 'calcium','20190625', 'S1'],
]

In [ ]:
config_file_list = []
for ix, session in enumerate(sessions_to_analyze):
    # name of the session config file
    config_file_list.append(path_to_default_config.replace('.yml', '_%s_%s_%s.yml' % 
                                                           (session[0], session[1], session[2])))
    # open default config file
    with open(path_to_default_config) as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    
    # replace session info
    config['data']['animal_folder'] = session[0]
    config['data']['day_folder'] = session[1]
    config['data']['area_folder'] = session[2]
    
    # check if session exists
    data_folder = os.path.join(config['data']['data_folder'], session[0], session[1], session[2])
    if not os.path.exists(data_folder):
        raise Exception('The folder %s could not be found!' % (data_folder))
    
    # save session config file
    with open(config_file_list[ix], 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)

In [ ]:
def delete_dirs_in_tmp():
    '''
    Function to try and delete tmp folders from a previous analysis
    '''
    for x in os.listdir('/tmp'):
        if x.startswith('tmp'):
            try:
                os.rmdir(os.path.join('/tmp', x))
                print('Deleted directory %s' % os.path.join('/tmp', x))
            except:
                print('Failed to delete directory %s' % os.path.join('/tmp', x))

In [ ]:
input_nb = '01_Preprocess_MC_3D.ipynb'
for ix, session in enumerate(sessions_to_analyze):
    # delete left-over directories from previous (failed )analysis
    delete_dirs_in_tmp()
    
    current_config = config_file_list[ix]
    output_nb = input_nb.replace('.ipynb', '_%s_%s_%s.ipynb' % (session[0], session[1], session[2]))
    print('Running motion correction for %s %s %s.\nOutput notebook: %s' % 
          (session[0], session[1], session[2], output_nb))
    
    try:
        pm.execute_notebook(
            input_nb, 
            output_nb, 
            parameters = dict(config_file=current_config)
        )
        print('Finished execution of notebook for %s %s %s' % (session[0], session[1], session[2]))
        with open(current_config.replace('.yml','.success'), 'w') as outfile:
            pass
    except:
        with open(current_config.replace('.yml','.error'), 'w') as outfile:
            pass
        raise Exception('Error during execution of notebook for %s %s %s. Check output notebook!' 
                        % (session[0], session[1], session[2]))
        break

    os.remove(current_config)